In [3]:
# Import libraries
import requests, json
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
# using Postman we found that the list comes from a json file
url = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'
r = requests.get(url)
universityJson = json.loads(r.text)

In [86]:
# Let's get the top 200 and display them
pd.set_option('display.max_columns', 500)
top200 = pd.DataFrame(universityJson['data'])[:200]
top200

,aliases,location,member_level,name,nid,rank,rank_order,record_type,scores_citations,scores_citations_rank,scores_industry_income,scores_industry_income_rank,scores_international_outlook,scores_international_outlook_rank,scores_overall,scores_overall_rank,scores_research,scores_research_rank,scores_teaching,scores_teaching_rank,stats_female_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,subjects_offered,url
0,University of Oxford,United Kingdom,0,University of Oxford,468,1,10,master_account,99.1,15,63.7,169,95.0,24,94.3,10,99.5,1,86.7,5,46 : 54,"20,409",38%,11.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-oxford
1,University of Cambridge,United Kingdom,0,University of Cambridge,470,2,20,master_account,97.5,29,51.5,260,93.0,35,93.2,20,97.8,3,87.8,3,45 : 55,"18,389",35%,10.9,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/university-cambridge
2,California Institute of Technology caltech,United States,0,California Institute of Technology,128779,=3,30,private,99.5,10,92.6,51,59.7,322,93.0,30,97.5,4,90.3,1,31 : 69,"2,209",27%,6.5,"Architecture,Biological Sciences,Business & Ma...",/world-university-rankings/california-institut...
3,Stanford University,United States,11,Stanford University,467,=3,40,private,99.9,4,60.5,189,77.6,162,93.0,40,96.7,5,89.1,2,42 : 58,"15,845",22%,7.5,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/stanford-university
4,Massachusetts Institute of Technology,United States,0,Massachusetts Institute of Technology,471,5,50,private,100.0,1,88.4,63,87.6,81,92.5,50,91.9,9,87.3,4,37 : 63,"11,177",34%,8.7,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/massachusetts-insti...
5,Harvard University,United States,0,Harvard University,466,6,60,private,99.7,8,46.4,330,79.7,143,91.8,60,98.4,2,84.2,9,None,"20,326",26%,8.9,"Agriculture & Forestry,Archaeology,Art, Perfor...",/world-university-rankings/harvard-university
6,Princeton University,United States,0,Princeton University,469,7,70,private,99.6,9,58.0,204,78.7,152,91.1,70,93.9,6,85.7,7,45 : 55,"7,955",24%,8.3,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/princeton-university
7,Imperial College London,United Kingdom,0,Imperial College London,472,8,80,master_account,96.7,40,71.6,120,96.6,13,89.2,80,88.7,12,81.7,13,37 : 63,"15,857",55%,11.4,"Biological Sciences,Chemical Engineering,Chemi...",/world-university-rankings/imperial-college-lo...
8,University of Chicago,United States,0,University of Chicago,473,9,90,private,99.4,12,39.8,487,69.6,231,88.6,90,90.1,10,85.3,8,44 : 56,"13,525",25%,6.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-chicago
9,ETH Zurich – Swiss Federal Institute of Techno...,Switzerland,0,ETH Zurich – Swiss Federal Institute of Techno...,479,=10,100,master_account,94.3,60,60.3,190,98.1,8,87.7,100,92.0,8,76.4,21,31 : 69,"19,233",38%,14.6,"Agriculture & Forestry,Architecture,Biological...",/world-university-rankings/eth-zurich-swiss-fe...


In [87]:
locationData = pd.DataFrame()
for index, row in top200.iterrows():
    nid = row['nid']
    #comparisonLink = 'https://www.timeshighereducation.com/sites/default/files/university/comparison/'+str(nid)+'.json'
    #rankingsLink = 'https://www.timeshighereducation.com/sites/default/files/university/rankings/'+str(nid)+'.json'
    locationLink = 'https://www.timeshighereducation.com/sites/default/files/institution_markers/gmap_'+str(nid)+'.json'
    rLocation = requests.get(locationLink)
    if rLocation.ok:
        locationJson = json.loads(rLocation.text)
        locationDF = pd.DataFrame(comparisonJson)
        locationData = locationData.append(locationDF, ignore_index=True)

In [88]:
locationData

,name,nid,rank,scores_citations,scores_industry_income,scores_international_outlook,scores_overall,scores_research,scores_teaching,stats_female_ratio,stats_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,url
0,Aalto University,767,190,79.5,52.5,72.5,52.7,35.8,37.8,34,66,12744,20,20.2,world-university-rankings/aalto-university
1,University of Massachusetts,528,191,88.8,51.0,54.8,52.6,32.2,36.3,49,51,59230,14,13.3,world-university-rankings/university-massachus...
2,University of Auckland,637,192,73.8,67.3,90.6,52.5,40.7,32.5,54,46,30348,29,18.8,world-university-rankings/university-auckland
3,Northeastern University,681,193,91.7,35.4,70.1,52.4,26.8,35.6,49,51,18780,32,14.0,world-university-rankings/northeastern-university
4,Lomonosov Moscow State University,753,194,15.8,89.0,61.1,52.3,61.6,74.2,48,52,29236,22,7.3,world-university-rankings/lomonosov-moscow-sta...
5,Tilburg University,734,195,61.3,59.0,75.8,52.1,50.1,38.4,48,52,7738,13,22.5,world-university-rankings/tilburg-university
6,Paris-Sorbonne University – Paris 4,924,196,77.1,32.1,57.9,52.0,34.3,44.6,68,32,22042,15,30.2,world-university-rankings/paris-sorbonne-unive...
7,"Royal Holloway, University of London",571,197,85.0,35.0,93.0,51.9,27.9,33.9,59,41,8931,40,15.4,world-university-rankings/royal-holloway-unive...
8,Aalto University,767,190,79.5,52.5,72.5,52.7,35.8,37.8,34,66,12744,20,20.2,world-university-rankings/aalto-university
9,University of Massachusetts,528,191,88.8,51.0,54.8,52.6,32.2,36.3,49,51,59230,14,13.3,world-university-rankings/university-massachus...


In [74]:
# Which are the best universities in term of: (a) ratio between faculty members and students
# The stats_student_staff_ratio values seem to be stored as str so let's fix it
top200.stats_student_staff_ratio = top200.stats_student_staff_ratio.apply(pd.to_numeric)
# Sort based on the ratio (lower better) and display top 10
top200.sort_values('stats_student_staff_ratio',ascending=True)[:10][['name','stats_student_staff_ratio']]

,name,stats_student_staff_ratio
105,Vanderbilt University,3.3
109,University of Copenhagen,4.1
12,Johns Hopkins University,4.3
11,Yale University,4.3
153,University of Rochester,4.3
97,Emory University,4.4
16,Duke University,4.5
114,École Polytechnique,5.1
183,Scuola Normale Superiore di Pisa,5.2
101,University of Pittsburgh,5.9


In [79]:
# Which are the best universities in term of: (b) ratio of international students
# The stats_pc_intl_students is stored as percentage, let's make it ratio
def p2f(x):
    return float(x.strip('%'))/100

top200.stats_pc_intl_students = top200.stats_pc_intl_students.apply(p2f)
# Sort based on the ratio (higher better) and display top 10
top200.sort_values('stats_pc_intl_students',ascending=False)[:10][['name','stats_pc_intl_students']]

,name,stats_pc_intl_students
24,London School of Economics and Political Science,0.71
178,University of Luxembourg,0.57
37,École Polytechnique Fédérale de Lausanne,0.55
7,Imperial College London,0.55
102,Maastricht University,0.50
15,University College London,0.49
143,University of St Andrews,0.48
23,Carnegie Mellon University,0.45
120,Queen Mary University of London,0.45
39,University of Hong Kong,0.42


In [66]:
type(top200.stats_student_staff_ratio.min())

str

In [20]:
r2 = requests.get('https://www.timeshighereducation.com/world-university-rankings/2018/world-ranking')

In [49]:
comparisonData = pd.DataFrame()
for index, row in top200.iterrows():
    nid = row['nid']
    comparisonLink = 'https://www.timeshighereducation.com/sites/default/files/university/comparison/'+str(nid)+'.json'
    #rankingsLink = 'https://www.timeshighereducation.com/sites/default/files/university/rankings/'+str(nid)+'.json'
    locationLink = 'https://www.timeshighereducation.com/sites/default/files/institution_markers/gmap_'+str(nid)+'.json'
    rComparison = requests.get(comparisonLink)
    rLocation = requests.get(locationLink)
    if rComparison.ok:
        comparisonJson = json.loads(rComparison.text)
        comparisonDF = pd.DataFrame(comparisonJson)
        comparisonDF = comparisonDF[comparisonDF['nid'] == nid]
        comparisonData = comparisonData.append(comparisonDF, ignore_index=True)
    

In [50]:
comparisonData

,name,nid,rank,scores_citations,scores_industry_income,scores_international_outlook,scores_overall,scores_research,scores_teaching,stats_female_ratio,stats_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,url
0,University of Oxford,468,1,99.1,63.7,95.0,94.3,99.5,86.7,46.0,54.0,20409,38,11.2,world-university-rankings/university-oxford
1,California Institute of Technology,128779,=3,99.5,92.6,59.7,93.0,97.5,90.3,31.0,69.0,2209,27,6.5,world-university-rankings/california-institute...
2,Harvard University,466,6,99.7,46.4,79.7,91.8,98.4,84.2,NaN,NaN,20326,26,8.9,world-university-rankings/harvard-university
3,Imperial College London,472,8,96.7,71.6,96.6,89.2,88.7,81.7,37.0,63.0,15857,55,11.4,world-university-rankings/imperial-college-london
4,University of Chicago,473,9,99.4,39.8,69.6,88.6,90.1,85.3,44.0,56.0,13525,25,6.2,world-university-rankings/university-chicago
5,ETH Zurich – Swiss Federal Institute of Techno...,479,=10,94.3,60.3,98.1,87.7,92.0,76.4,31.0,69.0,19233,38,14.6,world-university-rankings/eth-zurich-swiss-fed...
6,University of Pennsylvania,480,=10,98.5,56.9,61.3,87.7,90.1,83.7,50.0,50.0,20361,20,6.5,world-university-rankings/university-pennsylvania
7,Yale University,475,12,98.4,45.1,64.6,87.6,87.0,86.7,49.0,51.0,12155,21,4.3,world-university-rankings/yale-university
8,Johns Hopkins University,478,13,98.4,95.8,70.6,86.5,88.1,76.1,52.0,48.0,15498,24,4.3,world-university-rankings/johns-hopkins-univer...
9,"University of California, Los Angeles",477,15,97.9,48.6,59.5,85.7,88.1,80.7,53.0,47.0,39279,17,9.6,world-university-rankings/university-californi...


In [55]:
top200.columns.difference(comparisonData.columns)

Index(['aliases', 'location', 'member_level', 'rank_order', 'record_type',
       'scores_citations_rank', 'scores_industry_income_rank',
       'scores_international_outlook_rank', 'scores_overall_rank',
       'scores_research_rank', 'scores_teaching_rank',
       'stats_female_male_ratio', 'subjects_offered'],
      dtype='object')

In [46]:
print(comparisonLink)
print('https://www.timeshighereducation.com/sites/default/files/university/comparison/470.json')

https://www.timeshighereducation.com/sites/default/files/university/comparison/470.json
https://www.timeshighereducation.com/sites/default/files/university/comparison/470.json


In [47]:
rComparison.ok

False

In [80]:
nid = 468
comparisonLink = 'https://www.timeshighereducation.com/sites/default/files/university/comparison/'+str(nid)+'.json'
#rankingsLink = 'https://www.timeshighereducation.com/sites/default/files/university/rankings/'+str(nid)+'.json'
locationLink = 'https://www.timeshighereducation.com/sites/default/files/institution_markers/gmap_'+str(nid)+'.json'
rLocation = requests.get(locationLink)

In [83]:
locationJson = json.loads(rLocation.text)
comp = pd.DataFrame(locationJson)
comp

,country,lat,lng,locality,path,postal_code,thoroughfare,title
0,United Kingdom,51.757701873800,-1.263959050200,Oxford,node/468,OX1 2JD,University Offices,University of Oxford


In [28]:
pd.merge(top200,comp,on='name')

,aliases,location,member_level,name,nid_x,rank_x,rank_order,record_type,scores_citations_x,scores_citations_rank,scores_industry_income_x,scores_industry_income_rank,scores_international_outlook_x,scores_international_outlook_rank,scores_overall_x,scores_overall_rank,scores_research_x,scores_research_rank,scores_teaching_x,scores_teaching_rank,stats_female_male_ratio,stats_number_students_x,stats_pc_intl_students_x,stats_student_staff_ratio_x,subjects_offered,url_x,nid_y,rank_y,scores_citations_y,scores_industry_income_y,scores_international_outlook_y,scores_overall_y,scores_research_y,scores_teaching_y,stats_female_ratio,stats_male_ratio,stats_number_students_y,stats_pc_intl_students_y,stats_student_staff_ratio_y,url_y
0,University of Oxford,United Kingdom,0,University of Oxford,468,1,10,master_account,99.1,15,63.7,169,95.0,24,94.3,10,99.5,1,86.7,5,46 : 54,"20,409",38%,11.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-oxford,468,1,99.1,63.7,95.0,94.3,99.5,86.7,46.0,54.0,20409,38,11.2,world-university-rankings/university-oxford
1,University of Cambridge,United Kingdom,0,University of Cambridge,470,2,20,master_account,97.5,29,51.5,260,93.0,35,93.2,20,97.8,3,87.8,3,45 : 55,"18,389",35%,10.9,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/university-cambridge,470,2,97.5,51.5,93.0,93.2,97.8,87.8,45.0,55.0,18389,35,10.9,world-university-rankings/university-cambridge
2,California Institute of Technology caltech,United States,0,California Institute of Technology,128779,=3,30,private,99.5,10,92.6,51,59.7,322,93.0,30,97.5,4,90.3,1,31 : 69,"2,209",27%,6.5,"Architecture,Biological Sciences,Business & Ma...",/world-university-rankings/california-institut...,128779,=3,99.5,92.6,59.7,93.0,97.5,90.3,31.0,69.0,2209,27,6.5,world-university-rankings/california-institute...
3,Stanford University,United States,11,Stanford University,467,=3,40,private,99.9,4,60.5,189,77.6,162,93.0,40,96.7,5,89.1,2,42 : 58,"15,845",22%,7.5,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/stanford-university,467,=3,99.9,60.5,77.6,93.0,96.7,89.1,42.0,58.0,15845,22,7.5,world-university-rankings/stanford-university
4,Massachusetts Institute of Technology,United States,0,Massachusetts Institute of Technology,471,5,50,private,100.0,1,88.4,63,87.6,81,92.5,50,91.9,9,87.3,4,37 : 63,"11,177",34%,8.7,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/massachusetts-insti...,471,5,100.0,88.4,87.6,92.5,91.9,87.3,37.0,63.0,11177,34,8.7,world-university-rankings/massachusetts-instit...
5,Harvard University,United States,0,Harvard University,466,6,60,private,99.7,8,46.4,330,79.7,143,91.8,60,98.4,2,84.2,9,None,"20,326",26%,8.9,"Agriculture & Forestry,Archaeology,Art, Perfor...",/world-university-rankings/harvard-university,466,6,99.7,46.4,79.7,91.8,98.4,84.2,NaN,NaN,20326,26,8.9,world-university-rankings/harvard-university
6,Princeton University,United States,0,Princeton University,469,7,70,private,99.6,9,58.0,204,78.7,152,91.1,70,93.9,6,85.7,7,45 : 55,"7,955",24%,8.3,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/princeton-university,469,7,99.6,58.0,78.7,91.1,93.9,85.7,45.0,55.0,7955,24,8.3,world-university-rankings/princeton-university
7,Imperial College London,United Kingdom,0,Imperial College London,472,8,80,master_account,96.7,40,71.6,120,96.6,13,89.2,80,88.7,12,81.7,13,37 : 63,"15,857",55%,11.4,"Biological Sciences,Chemical Engineering,Chemi...",/world-university-rankings/imperial-college-lo...,472,8,96.7,71.6,96.6,89.2,88.7,81.7,37.0,63.0,15857,55,11.4,world-university-rankings/imperial-college-london
8,University of Chicago,United States,0,University of Chicago,473,9,90,private,99.4,12,39.8,487,69.6,231,88.6,90,90.1,10,85.3,8,44 : 56,"13,525",25%,6.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-chicago,473,9,99.4,39.8,69.6,88.6,90.1,85.3,44.0,56.0,13525,25,6.2,world-u